In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
from urllib.request import urlretrieve
import urllib.request
import os
import requests
import time
from random import uniform
import pandas as pd
# chromedriver로 크롬 사이트 접속하기
# 드라이버 경로는 본인 컴퓨터 경로로 수정해야함
driver = webdriver.Chrome('./chromedriver_win32/chromedriver.exe')
driver.implicitly_wait(3)
# 네이버 쇼핑윈도 - 원피스 카테고리로 들어가기
driver.get('https://search.shopping.naver.com/search/category.nhn?pagingIndex=1&pagingSize=40&productSet=window&viewType=list&sort=rel&cat_id=50000807&frm=NVSHCAT')
# csv 파일 가져오기
import csv
file='./0_1999.csv'#파일이름
f = open(file,'rt')
link_addr_file = csv.reader(f)
link_list=[]
for line in link_addr_file:
    link_list.append(line[1])
f.close()    
len(link_list)
link_list
#상세 페이지 들어가서 원하는 정보 저장
data_list = pd.DataFrame()
#link_list 에서 범위 조정 가능
for i in link_list:    
    driver.get(i)
    print(i)
    
    f = uniform(1.5, 1.7)
    time.sleep(f)

    a = ""
    a = i.split("_mid=")
    a[1][0:11]
    link = i

    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser') 

    try:
        images = soup.select('div._396uGDX0Fi > img')
        name = soup.find('h3','_2IA5sp7BRM').get_text()
        fit = driver.find_element_by_css_selector('div._1Hbih69XFT > div:nth-child(4) > table > tbody > tr:nth-child(1) > td:nth-child(2)').text        
        sleeve = driver.find_element_by_css_selector('div._1Hbih69XFT > div:nth-child(4) > table > tbody > tr:nth-child(2) > td:nth-child(2)').text
        length = driver.find_element_by_css_selector('div._1Hbih69XFT > div:nth-child(4) > table > tbody > tr:nth-child(3) > td:nth-child(2)').text
        pattern = driver.find_element_by_css_selector('div._1Hbih69XFT > div:nth-child(4) > table > tbody > tr:nth-child(4) > td:nth-child(2)').text
        material = driver.find_element_by_css_selector('div._1Hbih69XFT > div:nth-child(4) > table > tbody > tr:nth-child(1) > td:nth-child(4)').text
        neckline = driver.find_element_by_css_selector('div._1Hbih69XFT > div:nth-child(4) > table > tbody > tr:nth-child(2) > td:nth-child(4)').text
        skirt = driver.find_element_by_css_selector('div._1Hbih69XFT > div:nth-child(4) > table > tbody > tr:nth-child(3) > td:nth-child(4)').text
        detail = driver.find_element_by_css_selector('div._1Hbih69XFT > div:nth-child(4) > table > tbody > tr:nth-child(4) > td:nth-child(4)').text
        
#         price = soup.find('span','_1dpDPg-OQb').get_text()
#         tag=soup.find('div','_24FKuaBdoK').get_text()
#         info = [soup.find_all('th','_15qeGNn6Dt')[n].string for n in range(0,20)]
#         detail = [soup.find_all('td','jvlKiI0U_y')[n].string for n in range(0,20)]
#         texts = soup.find_all('p', 'se_textarea')

        if not os.path.isdir(a[1][0:11]):
            os.mkdir(a[1][0:11])     
        for image in images:

            src_page = requests.get(image['src'], headers={"Referer":"https://www.naver.com/"})
            src_image = src_page.content
            with open(".\\"+a[1][0:11]+"\\img_"+a[1][0:11]+'_'+str(images.index(image))+".png", "wb") as downfile:
                downfile.write(src_image)
            f = uniform(0.4, 0.6)    
            time.sleep(f)
#         for text in texts:
#             text_list.append(text.get_text().strip().replace(u'\xa0', u' '))
#             text_list2 = [zz for zz in text_list if zz]

        # 긁어올 카테고리 명을 입력하세요     
        data_1 = {'name': [name], 'link': [link], 'fit': [fit], 'sleeve': [sleeve], 'length': [length], 'pattern': [pattern], 
                  'material': [material], 'neckline': [neckline], 'skirt': [skirt], 'detail': [detail], 'imagefile':["img_"+a[1][0:11]+'_'+str(images.index(image))+".png"]}

    
#         data_1 = ({'name': [name], 'link': [link], 'fit': [fit], 'sleeve': [sleeve], 'length': [length],
#                   'pattern': [pattern], 'material': [material], 'neckline': [neckline], 'skirt': [skirt], 'detail': [detail]})
        df1 = pd.DataFrame(data_1)
        
#         data_2 = [detail]
#         df2 = pd.DataFrame(data_2, columns = info)

        data_list= pd.concat([data_list,df1], axis=0)

        df1.to_csv(".\\"+a[1][0:11]+"\\csv_"+a[1][0:11]+".csv", index = True, encoding='utf-8-sig')
    except:
        print("상품정보 가져오기 오류발생")
        pass
data_list.to_csv(file[0:-3]+"_list.csv", index=True, encoding='utf-8-sig')
